<a href="https://colab.research.google.com/github/Sogo95/Resume-de-texte/blob/main/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objectif :
Mettre en pratique les concepts de modèle encoder-decoder en construisant un modèle capable de générer un résumé automatique d'un texte.


1.	Téléchargez et préparez un jeu de données : Utilisez un jeu de données contenant des textes longs ainsi que leurs résumés. Par exemple, vous pouvez utiliser le jeu de données CNN/Daily Mail disponible sur Hugging Face Datasets. Préparez les données en suivant ces étapes :
•	Charger les textes et les résumés associés.
•	Effectuer un prétraitement (nettoyage du texte, suppression des caractères spéciaux, etc.).
•	Diviser les données en ensembles d’entraînement, de validation et de test.


In [15]:
pip install datasets

Charger les textes et les résumés associés.

In [16]:
import os
from datasets import load_dataset

# Charger le jeu de données CNN/Daily Mail
ds = load_dataset("abisee/cnn_dailymail", '3.0.0')


In [17]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [18]:
ds.shape

{'train': (287113, 3), 'validation': (13368, 3), 'test': (11490, 3)}

In [19]:
print(ds['train'][0]['article'])  # Texte de l'article
print(ds['train'][0]['highlights'])  # Résumé (highlights)

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

Effectuer un prétraitement (nettoyage du texte, suppression des caractères spéciaux, etc.

In [20]:
import re
from nltk.stem import WordNetLemmatizer
import nltk
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Télécharger les ressources pour la lemmatisation
nltk.download('wordnet')
nltk.download('punkt')

# Initialiser le lemmatizer
lemmatizer = WordNetLemmatizer()

# Fonction de nettoyage et lemmatisation
def netoy_lemmat(text):
    # Supprimer les balises HTML
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text)  # Supprimer les espaces multiples
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer les caractères spéciaux
    tokens = nltk.word_tokenize(text)  # Tokenisation
    lemmatized = [lemmatizer.lemmatize(token.lower()) for token in tokens]  # Lemmatisation
    return ' '.join(lemmatized)



# Appliquer le nettoyage et la lemmatisation
ds = ds.map(lambda x: {'article': netoy_lemmat(x['article']),
                        'highlights': netoy_lemmat(x['highlights'])})

print("Traitement  terminés.")


Traitement  terminés.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Diviser les données en ensembles d’entraînement, de validation et de test.

In [21]:
# Mélanger les données pour assurer un échantillonnage aléatoire
ds['train'] = ds['train'].shuffle(seed=42)
ds['validation'] = ds['validation'].shuffle(seed=42)
ds['test'] = ds['test'].shuffle(seed=42)

# Sélectionner 10% des données de chaque ensemble
train_sample = ds['train'].select(range(int(0.1 * len(ds['train']))))
val_sample = ds['validation'].select(range(int(0.1 * len(ds['validation']))))
test_sample = ds['test'].select(range(int(0.1 * len(ds['test']))))

print(f"Ensemble d'entraînement échantillonné: {len(train_sample)} exemples")
print(f"Ensemble de validation échantillonné: {len(val_sample)} exemples")
print(f"Ensemble de test échantillonné: {len(test_sample)} exemples")


Ensemble d'entraînement échantillonné: 28711 exemples
Ensemble de validation échantillonné: 1336 exemples
Ensemble de test échantillonné: 1149 exemples


    Construction du modèle Encoder-Decoder : Utilisez une architecture Transformer basée sur un modèle pré-entraîné tel que BART ou T5. Ces modèles sont particulièrement adaptés pour les tâches de résumé de texte.

    L'encoder reçoit le texte original et transforme les séquences d'entrée en une représentation cachée.
    Le decoder génère un résumé à partir de cette représentation cachée.

In [22]:
from transformers import GenerationConfig

# Créez un fichier GenerationConfig avec les paramètres personnalisés
generation_config = GenerationConfig(
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)

# Sauvegardez ce fichier
generation_config.save_pretrained('path_to_save_config')


Charger le modèle pré-entraîné et le tokenizer



In [23]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer  # Pour BART
# from transformers import T5ForConditionalGeneration, T5Tokenizer  # Pour T5

import torch
from transformers import BartForConditionalGeneration, BartTokenizer

# Initialiser le tokenizer et le modèle BART
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Déplacer le modèle vers le GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

Préparation des données pour l'entrainement

In [24]:
# Fonction de tokenisation
def tokenize_function(examples):
    inputs = tokenizer(examples['article'], truncation=True, padding='max_length', max_length=512)
    targets = tokenizer(examples['highlights'], truncation=True, padding='max_length', max_length=150)
    inputs['labels'] = targets['input_ids']
    return inputs

In [25]:

# Appliquer la tokenisation sur les ensembles d'entraînement et de validation
train_sample = train_sample.map(tokenize_function, batched=True)
val_sample = val_sample.map(tokenize_function, batched=True)

# Vérifier les données tokenisées
print(train_sample[0])


{'article': 'by anthony bond published est march updated est march three member of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious within minute investigator said today the body of married couple john and audrey cook were discovered alongside their daughter maureen at the mobile home they shared on tremarle home park in camborne west cornwall the inquest have now opened into the death last saturday with investigator saying the three died along with the family pet dog of carbon monoxide poisoning from a cooker tragic the inquest have opened into the death of three member of the same family who were found in their static caravan last weekend john and audrey cook are pictured awful the family died following carbon monoxide poisoning at this caravan at the tremarle home park in camborne cornwall it is also believed there wa no working carbon monoxide detector in the static caravan cornwall fire and rescue service said this would have 

Entrainement du modèle

In [14]:
from transformers import Trainer, TrainingArguments
# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',          # Évaluer après chaque époque
    learning_rate=3e-5,                   # Taux d'apprentissage
    per_device_train_batch_size=4,        # Taille du lot pour l'entraînement
    per_device_eval_batch_size=4,         # Taille du lot pour la validation
    num_train_epochs=3,                   # Nombre d'époques
    weight_decay=0.01,                    # Pénalité de poids
    save_total_limit=3,

)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_sample,
    eval_dataset=val_sample,
)

# Entraîner le modèle
trainer.train()

print("Entraînement terminé.")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.753300,0.803024
2,0.558700,0.800872
3,0.444300,0.833085


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

Entraînement terminé.


Évaluation : Une fois le modèle entraîné, évaluez ses performances en générant des résumés pour les textes du jeu de test. Utilisez des métriques comme ROUGE ou BLEU pour évaluer la qualité des résumés générés par rapport aux résumés de référence.

In [29]:
# Fonction pour générer des résumés pour les textes du jeu de test
def generate_summaries(test_sample, model, tokenizer, max_length=1024, num_beams=4):
    model.eval()
    summaries = []
    for sample in test_sample:
        inputs = tokenizer(sample['article'], return_tensors="pt", truncation=True, padding="max_length", max_length=max_length).to(device)
        summary_ids = model.generate(inputs["input_ids"], num_beams=num_beams, max_length=150, early_stopping=True)
        summaries.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False))
    return summaries


In [33]:
# Supposons que 'test_sample' contient les textes du jeu de test
summaries = generate_summaries(test_sample, model, tokenizer)


In [34]:
for i, sample in enumerate(test_sample):
    print(f"Texte original {i + 1}:")
    print(sample['article'])  # Access the correct key 'article' instead of 'text'
    print("\nRésumé généré:")
    print(summaries[i])  # Affiche le résumé correspondant
    print("\n" + "="*50 + "\n")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Texte original 525:
cnnthe commissionergeneral of the united nation relief and work agency will make an emergency visit to the yarmouk palestinian refugee camp in syria on saturday a spokesman say commissionergeneral pierre krhenbhl will ass the humanitarian situation in the camp and speak with individual about way to relieve the suffering of the people who remain there the visit is prompted by unrwas deepening concern for the safety and protection of palestinian and syrian civilian including child agency spokesman christopher gunness told cnns paula newton yarmouk remains under the control of armed group and civilian life continues to be threatened by the effect of the conflict krhenbhl will meet with senior syrian official un and relief agency staff member and displaced people from the camp itself the yarmouk refugee camp which sits just mile from central damascus ha been engulfed in fighting between the sy

In [54]:
 # Génération des résumés pour le jeu de test

 test_summaries = generate_summaries(test_sample, model, tokenizer)

 Calcul des métriques ROUGE ou BLEU

In [47]:
# Installer la bibliothèque evaluate (si non déjà installé)
!pip install evaluate

In [49]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=da2d32b45b1f67b6b66be9d6d838d6d75eedf2be49796cbd19f0caff20fbdc6b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [55]:


# Importer la bibliothèque evaluate
import evaluate

# Charger la métrique ROUGE
rouge_metric = evaluate.load("rouge")

# Comparaison des résumés générés avec les résumés de référence
references = [sample['article'] for sample in test_sample]
rouge_scores = rouge_metric.compute(predictions=test_summaries, references=references)

# Affichage des résultats ROUGE
print("Scores ROUGE:", rouge_scores)



Scores ROUGE: {'rouge1': 0.23317516030102442, 'rouge2': 0.222284529867686, 'rougeL': 0.22700844860602995, 'rougeLsum': 0.22677143270359817}


In [ ]:

# Charger la métrique BLEU
bleu_metric = evaluate.load("bleu")

# Pour BLEU, les résumés de référence doivent être sous forme de liste de listes de mots
references_bleu = [[ref.split()] for ref in references]  # Chaque référence doit être une liste de listes de mots
predictions_bleu = [pred.split() for pred in test_summaries]  # Les prédictions doivent être sous forme de listes de mots

# Calcul des scores BLEU
bleu_score = bleu_metric.compute(predictions=predictions_bleu, references=references_bleu)

# Affichage du score BLEU
print("Score BLEU:", bleu_score)
